In [ ]:
import numpy as np
import gymu
import jnu as J

env = gymu.make("thesis/MNIST-v0", num_actions=4)  
print(env.spec)

In [ ]:

iterator = gymu.iterator(env, mode=gymu.mode.sa, max_length=1000)

dataset = gymu.data.dataset(iterator)
dataset = dataset.gymu.mode(gymu.mode.sa)
dataset = dataset.gymu.to_tensor_dataset()

s, a = dataset.tensors
J.images(s, scale=3, on_interact=np.array(env.get_action_meanings())[a.numpy()])


In [ ]:
from gym import envs
print(envs.registry.all())

In [ ]:
import pathlib
import jnu as J
import gymu

# load data

env_id = "ALE/Breakout-v5"

path = pathlib.Path(f"../data/{env_id}").resolve()
urls = [str(p) for p in path.iterdir()]

dataset = gymu.data.dataset(urls)
dataset = dataset.gymu.decode(keep_meta=False)
dataset = dataset.gymu.mode(gymu.mode.sa)
dataset = dataset.gymu.to_tensor_dataset()

s, a = dataset.tensors

J.images(s, on_interact=np.array(env.get_action_meanings())[a.numpy()])


In [ ]:
import jnu as J
import gymu
import numpy as np

env_id = "ALE/Breakout-v5"

env = gymu.make(env_id, 
                obs_type='rgb', 
                frameskip=1, 
                mode=0, 
                difficulty=0, 
                repeat_action_probability=0, 
                full_action_space=False, 
                render_mode=None)

class FireReset(gym.Wrapper):

    def reset(self):
        self.env.reset()
        fire = self.env.get_action_meanings().index("FIRE")
        state, *_ = self.env.step(fire)
        return state
    
env = FireReset(env)

iterator = gymu.iterator(env, lambda x : 0, mode=gymu.mode.sa, max_length=1000)

import torchvision.transforms as T
preprocess = T.Compose([
    T.ToTensor(),
    #T.Resize((64,64)),
])


dataset = gymu.data.dataset(iterator).map(lambda x: dict(**x))
dataset = dataset.map_dict(state=preprocess)
dataset = dataset.gymu.mode(gymu.mode.sa)
dataset = dataset.gymu.to_tensor_dataset()

s, a = dataset.tensors
J.images(s, scale=3, on_interact=np.array(env.get_action_meanings())[a.numpy()])


